In [ ]:
"""
changelogs:
    head:
        change output to ReLU
"""

In [1]:
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

In [2]:
model_name = 'm204_ReLU2'
writer = SummaryWriter(f'logs_tb/{model_name}', filename_suffix='-model_graph.tb')

In [3]:
def calc_padding(kernel_size, padding):
    if padding == 'same':
        return kernel_size//2
    elif padding == 'valid':
        return 0

def conv1d(Ci, Co, kernel_size, stride, padding):
    module = nn.Conv1d(Ci, Co,
                       kernel_size=kernel_size,
                       stride=stride,
                       padding=calc_padding(kernel_size, padding))
    nn.init.kaiming_normal_(module.weight) # He normal
    return module

def conv1d_relu(Ci, Co, kernel_size, stride, padding='same'):
    module = nn.Sequential(
        conv1d(Ci, Co, kernel_size, stride, padding),
        nn.ReLU()
    )
    return module

def CNN_Block(Ci, Co):
    module = nn.Sequential(
        conv1d_relu(Ci, Co, 3, 1),
        conv1d_relu(Co, Co, 3, 1),
        conv1d_relu(Co, Co, 2, 2)
    )
    return module

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Sequential(
            conv1d_relu(1, 16, 11, 6, 'valid'),
            conv1d_relu(16, 16, 3, 2, 'valid'),
            conv1d_relu(16, 16, 3, 2, 'valid')
        )
        self.body = nn.Sequential(
            CNN_Block(16, 32),
            CNN_Block(32, 64),
            CNN_Block(64, 64)
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.Flatten(),
            nn.Linear(64, 16),
            nn.Sigmoid(),
            nn.Linear(16, 1),
            nn.ReLU()
        )
        
    def forward(self, x):
        y = self.stem(x)
        y = self.body(y)
        y = self.head(y)
        y = torch.clamp(y, max=1.)
        return y
    
model = Model().cuda()
model

Model(
  (stem): Sequential(
    (0): Sequential(
      (0): Conv1d(1, 16, kernel_size=(11,), stride=(6,))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(16, 16, kernel_size=(3,), stride=(2,))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv1d(16, 16, kernel_size=(3,), stride=(2,))
      (1): ReLU()
    )
  )
  (body): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
      )
      (2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(2,), stride=(2,), padding=(1,))
        (1): ReLU()
      )
    )
    (1): Sequential(
      (0): Sequential(
        (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))

In [4]:
duration = 8*60*60 # 8 hr
SR = 1 # 1 hz
data_len = duration*SR

input_data = torch.ones(1, 1, data_len).cuda() # [batch, channel, row]
input_data.size()

torch.Size([1, 1, 28800])

In [5]:
writer.add_graph(model, input_data)
writer.close()